<a href="https://colab.research.google.com/github/HAL22/Kaggle-Competitions/blob/contradictory/Contradictory_My_Dear_Watson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install kaggle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
!pip install datasets transformers[sentencepiece]
!apt install git-lfs
from transformers import AutoTokenizer
import pandas as pd
from datasets import load_dataset
from datasets import Dataset, DatasetDict, load_dataset
from sklearn.model_selection import train_test_split
from transformers import TFAutoModelForSequenceClassification

In [9]:
%%capture
!pip install -q googletrans==3.1.0a0
from googletrans import Translator
from dask import bag, diagnostics # extends common interfaces like NumPy, Pandas, or Python iterators to larger-than-memory or distributed environments

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!git config --global user.email "thethelafaltein@gmail.com"
!git config --global user.name "HAL22"

In [5]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
%%capture
!unzip '/content/test.csv.zip'
!unzip '/content/train.csv.zip'

In [7]:
df_train = pd.read_csv("/content/train.csv")
df_test = pd.read_csv("/content/test.csv")

In [8]:
df_train.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


Translation

In [10]:
translations_csv = '/kaggle/working/train_translations.csv'

dest_choices = {'ar': 'Arabic', 'fr': 'French', 'sw': 'Swahili', 'ur': 'Urdu', 'vi': 'Vietnamese', 'ru': 'Russian', 'hi': 'Hindi', 'el': 'Greek', 'th': 'Thai', 'es': 'Spanish', 'de': 'German', 'tr': 'Turkish', 'bg': 'Bulgarian'}

In [11]:
def translate(words, dest):
            translator = Translator()
            decoded = translator.translate(words, dest=dest).text
            return decoded

In [14]:
# Example
translate("How are you","fr")

'Comment allez-vous'

In [15]:
def translate_job(df, mode='en'):
   premise_bag = bag.from_sequence(df.premise.tolist())
   hypothesis_bag = bag.from_sequence(df.hypothesis.tolist())

   if mode == 'en':
     df['translate_to_abv'] = 'en'
     df['translate_to'] = 'English'
     dest_bag = bag.from_sequence(df.translate_to_abv.tolist())
   elif mode == 'random':
     df['translate_to_abv'] = np.random.choice(list(dest_choices.keys()), df.shape[0])
     df['translate_to'] = df.translate_to_abv.map(dest_choices)
     dest_bag = bag.from_sequence(df.translate_to_abv.tolist())

   with diagnostics.ProgressBar():
     premise_translated = premise_bag.map(translate, dest=dest_bag).compute()
     hypothesis_translated = hypothesis_bag.map(translate, dest=dest_bag).compute()

   df['premise'] = premise_translated
   df['hypothesis'] = hypothesis_translated
   df['id'] = df['id'].astype(str) + '_' + df['translate_to_abv']
   df['lang_abv'] = df['translate_to_abv']
   df['language'] = df['translate_to']
   df['augmented'] = True
   df.drop('translate_to_abv', axis=1, inplace=True)
   df.drop('translate_to', axis=1, inplace=True)

   return df   

In [16]:
eng = df_train.loc[df_train.lang_abv == "en"].copy().pipe(translate_job, mode='random')

[########################################] | 100% Completed |  9min 18.9s
[########################################] | 100% Completed |  9min 30.7s


In [17]:
eng

,id,premise,hypothesis,lang_abv,language,label,augmented
0,5130fd2cb5_ar,وتم أخذ هذه التعليقات في الاعتبار عند صياغة ال...,تم وضع القواعد التي تم وضعها في غضون ذلك مع وض...,ar,Arabic,0,True
1,5b72532a0b_hi,उन्होंने कहा कि ये ऐसे मुद्दे हैं जिनसे हम कान...,अभ्यास समूहों को इन मुद्दों पर काम करने की अनु...,hi,Hindi,2,True
3,5622f0c60b_el,Ξέρεις ότι δεν μπορούν πραγματικά να υπερασπισ...,Δεν μπορούν να αμυνθούν λόγω της ηλικίας τους.,el,Greek,0,True
7,fdcd1bd867_sw,Kutoka Nchi ya Cockpit hadi St. Ann's Bay,Kutoka St. Ann's Bay hadi Cockpit Country.,sw,Swahili,2,True
8,7cfb3d272c_de,"Hören Sie, es ist Ihre Haut, aber Sie werden i...","Der Boss wird dich feuern, wenn er sieht, dass...",de,German,1,True
...,...,...,...,...,...,...,...
12115,2b78e2a914_el,Τα αποτελέσματα ακόμη και των πιο καλά σχεδιασ...,Όλες οι μελέτες έχουν την ίδια αβεβαιότητα.,el,Greek,2,True
12116,7e9943d152_ar,ولكن هناك نوعان من لذة الفعل ولذة عدم الفعل ؛ ...,ولكن هناك نوعان من لذة الفعل ولذة عدم القيام به.,ar,Arabic,0,True
12117,5085923e6c_ar,الشيء المهم هو إدراك أن الوقت قد حان لنقله.,لا يمكن نقله الآن أو في أي وقت مضى.,ar,Arabic,2,True
12118,fc8e2fd1fe_el,Στο δυτικό άκρο υπάρχει λεπτομερές ομοίωμα όλο...,Το πρότυπο συγκρότημα του ναού βρίσκεται στο α...,el,Greek,2,True


In [ ]:
to_eng = df_train.loc[df_train.lang_abv != "en"].copy().pipe(translate_job, mode='en')

[########################################] | 100% Completed |  7min 21.1s
[                                        ] | 0% Completed | 21.4s

In [ ]:
df_train['augmented'] = False
df_train = df_train.append([eng])
df_train = df_train.append([to_eng])
df_train.reset_index(drop=True, inplace=True)
df_train.to_csv(translations_csv, index=False)

In [8]:
df_train = df_train[["premise", "hypothesis","label"]].copy()

In [9]:
df_train.head()

,premise,hypothesis,label
0,and these comments were considered in formulat...,The rules developed in the interim were put to...,0
1,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,2
2,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,0
3,you know they can't really defend themselves l...,They can't defend themselves because of their ...,0
4,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,1


In [10]:
train = Dataset.from_pandas(df_train)
train

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 12120
})

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")


In [12]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased",num_labels=3,hidden_dropout_prob=0.35,attention_probs_dropout_prob=0.35,)

In [13]:
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["premise"],data["hypothesis"])

In [14]:
# For GPU
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", 
    hidden_dropout_prob=0.35, # The dropout probability for all fully connected layers in the embeddings, encoder, and pooler.
    attention_probs_dropout_prob=0.35, # The dropout ratio for the attention probabilities.
    num_labels=3,
).to(device) # The model uses cpu or gpu for training

KeyError: ignored

In [ ]:
tokenized_ds = train.map(tokenize_dataset, batched=True)

  0%|          | 0/13 [00:00<?, ?ba/s]

In [ ]:
# Transformers assumes that labels have the column name "labels"
tokenized_ds = tokenized_ds.rename_columns({"label": "labels"})

In [ ]:
dsd = tokenized_ds.train_test_split(.25, seed=42)
dsd

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9090
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3030
    })
})

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(pred):
    labels=pred.label_ids
    preds=pred.predictions.argmax(-1)
    f1=f1_score(labels, preds, average='weighted')
    ac=accuracy_score(labels, preds)
    return {"accuracy":ac, "f1":f1}

In [ ]:
from transformers import TrainingArguments, Trainer

bs = 16
epochs = 1
lr = 4e-5

In [ ]:
args = TrainingArguments(
    output_dir="./results",
    learning_rate=lr, 
    warmup_ratio=0.1, 
    lr_scheduler_type='cosine', 
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8, 
    weight_decay=0.01,)

trainer = Trainer(
    model=model, 
    args=args, 
    train_dataset=dsd['train'], 
    eval_dataset=dsd['test'],
    tokenizer=tokenizer, 
    compute_metrics=compute_metrics)

In [ ]:

torch.cuda.empty_cache()

In [ ]:
trainer.train();

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.492500
1000,0.623000
1500,0.888900
2000,0.849000
2500,0.780900
3000,0.730100
3500,0.694600
4000,0.640800
4500,0.611100
